In [1]:
#!pip install torch numpy matplotlib
#!pip install nest-asyncio

In [2]:

import asyncio
import nest_asyncio
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import random

# 환경 관련 import
from env.battle_env import YakemonEnv

# 모델 관련 import

# 유틸리티 관련 import

# 에이전트 관련 import
from agent.dddqn_agent import DDDQNAgent

# RL 관련 import

# 데이터 관련 import=

# 컨텍스트 관련 import
from context.battle_store import store
from context.duration_store import duration_store

In [3]:
# 전역 변수 초기화
battle_store = store
duration_store = duration_store

# 하이퍼파라미터 설정
HYPERPARAMS = {
    "learning_rate": 0.0005,
    "gamma": 0.95,
    "epsilon_start": 1.0,
    "epsilon_end": 0.01,
    "epsilon_decay": 0.997,
    "batch_size": 128,
    "memory_size": 50000,
    "target_update": 20,
    "num_episodes": 100,
    "save_interval": 50,
    "test_episodes": 50,
    "state_dim": 126,  # get_state_vector의 출력 차원
    "action_dim": 6,   # 4개의 기술 + 2개의 교체
    "memory_size": 50000,  # 리플레이 버퍼 크기
    "batch_size": 128,     # 배치 크기
}

In [4]:
from training_dqn import test_agent
from training_dqn import train_agent
# 메인 실행 코드
from datetime import datetime
from training_dqn import plot_training_results

In [5]:
# 메인 실행 코드
if __name__ == "__main__":
    # Jupyter에서 중첩된 이벤트 루프 허용
    nest_asyncio.apply()
    
    # 결과 저장 디렉토리 설정
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_dir = os.path.join('results', f'training_{timestamp}')
    models_dir = os.path.join('models', f'training_{timestamp}')
    
    # 환경 초기화
    env = YakemonEnv()  # 실제 게임 환경
    state_dim = HYPERPARAMS["state_dim"]
    action_dim = HYPERPARAMS["action_dim"]
    
    # DDDQN 에이전트 생성
    ddqn_agent = DDDQNAgent(
        state_dim=state_dim,
        action_dim=action_dim,
        learning_rate=HYPERPARAMS["learning_rate"],
        gamma=HYPERPARAMS["gamma"],
        epsilon_start=HYPERPARAMS["epsilon_start"],
        epsilon_end=HYPERPARAMS["epsilon_end"],
        epsilon_decay=HYPERPARAMS["epsilon_decay"],
        target_update=HYPERPARAMS["target_update"],
        memory_size=HYPERPARAMS["memory_size"],
        batch_size=HYPERPARAMS["batch_size"]
    )
    
    print("Starting DDDQN training...")
    print(f"Results will be saved in: {results_dir}")
    print(f"Models will be saved in: {models_dir}")
    print("\nHyperparameters:")
    for key, value in HYPERPARAMS.items():
        print(f"  {key}: {value}")
    print("\n" + "="*50 + "\n")
    
    # DDDQN 에이전트 학습
    ddqn_rewards, ddqn_losses = asyncio.run(train_agent(
        env=env,
        agent=ddqn_agent,
        num_episodes=HYPERPARAMS["num_episodes"],
        save_path=models_dir,
        agent_name='ddqn'
    ))
    
    # 학습 결과 시각화
    plot_training_results(
        rewards_history=ddqn_rewards,
        losses_history=ddqn_losses,
        agent_name='DDDQN',
        save_path=results_dir
    )
    
    print("\nTraining completed!")
    print(f"Results saved in: {results_dir}")
    print(f"Models saved in: {models_dir}")
    
    # 학습된 에이전트 테스트
    print("\nStarting test phase...")
    test_results = asyncio.run(test_agent(
        env=env,
        agent=ddqn_agent,
        num_episodes=HYPERPARAMS["test_episodes"]
    ))
    
    # 테스트 결과 저장
    test_stats = {
        'avg_reward': test_results[0],
        'std_reward': test_results[1],
        'avg_steps': test_results[2],
        'victories': test_results[3],
        'win_rate': test_results[4]
    }
    
    with open(os.path.join(results_dir, 'test_results.json'), 'w') as f:
        json.dump(test_stats, f, indent=4)
    
    with open(os.path.join(results_dir, 'test_results.txt'), 'w') as f:
        f.write("Test Results\n")
        f.write("=" * 50 + "\n\n")
        f.write(f"Average Reward: {test_stats['avg_reward']:.4f} ± {test_stats['std_reward']:.4f}\n")
        f.write(f"Average Steps: {test_stats['avg_steps']:.2f}\n")
        f.write(f"Victories: {test_stats['victories']}/{HYPERPARAMS['test_episodes']} (Win Rate: {test_stats['win_rate']:.1f}%)\n")
    
    print("\nTest completed!")
    print(f"Test results saved in: {results_dir}")



Starting DDDQN training...
Results will be saved in: results\training_20250517_200036
Models will be saved in: models\training_20250517_200036

Hyperparameters:
  learning_rate: 0.0005
  gamma: 0.95
  epsilon_start: 1.0
  epsilon_end: 0.01
  epsilon_decay: 0.997
  batch_size: 128
  memory_size: 50000
  target_update: 20
  num_episodes: 100
  save_interval: 50
  test_episodes: 50
  state_dim: 126
  action_dim: 6


[Episode 1] My Team (BattlePokemon):
{'base': <p_models.pokemon_info.PokemonInfo object at 0x00000239118E1A60>, 'current_hp': 153, 'pp': {'화염방사': 15, '지진': 10, '오버히트': 5, '원시의힘': 5}, 'rank': {'attack': 0, 'sp_attack': 0, 'defense': 0, 'sp_defense': 0, 'speed': 0, 'accuracy': 0, 'dodge': 0, 'critical': 0}, 'status': [], 'position': None, 'is_active': False, 'locked_move': None, 'locked_move_turn': None, 'is_protecting': False, 'used_move': None, 'had_missed': False, 'had_rank_up': False, 'is_charging': False, 'charging_move': None, 'received_damage': None, 'is_first_turn': Fals